<a href="https://colab.research.google.com/github/Jamie228/ump/blob/main/create_pascal_voc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Create PASCAL-VOC Annotation File from Annotation CSV
This code should take the CSV of bounding boxes for the dataset and create a Pascal-VOC XML file for each image. This will enable the dataset to be used with more object detection tools.

## Import Dependencies
The dependencies for this project are as follows:
- **Pandas**: For CSV importing and data queries
- **cElementTree**: For forming the XML files
- **NumPy**: For better iterability of the Pandas DataFrame

In [2]:
import pandas as pd
import xml.etree.cElementTree as ET
import numpy as np

## Define Constants and Important Variables

In [3]:
## Array of all image prefixes
images = ['A', 'B', 'D', 'E', 'F', 'G', 'LC', 'HC', 'XA', 'XB', 'XC', 'XD', 'XE', 'XF', 'XG', 'XH', 'XI', 'XJ']
## Path to CSV
CSV_PATH = './drive/MyDrive/Tensorflow/workspace/training_demo/images/train/labels.csv'
CSV_COLS = ['class', 'xmin', 'ymin', 'xmax', 'ymax', 'filename', 'width', 'height']

## Import CSV and Label Columns

In [4]:
annotations = pd.read_csv(CSV_PATH, header=None, names=CSV_COLS)
annotations.head()

,class,xmin,ymin,xmax,ymax,filename,width,height
0,A,10,8,28,70,A.jpeg,41,78
1,A,8,5,27,69,A_1.jpeg,41,78
2,A,14,2,31,67,A_2.jpeg,41,78
3,A,7,9,29,71,A_3.jpeg,41,78
4,A,8,6,28,71,A_4.jpeg,41,78


In [ ]:
img = "A"
fn = img + ".jpeg"
ann = annotations.query("filename == @fn")
ann = np.array(ann)
width = ann[0][6]
height = ann[0][7]
print(width, height)

41 78


## Define XML Creation Function

In [7]:
def create_xml(img):
  """Creates an XML file for a given image prefix

  Arguments:
  img (str): A string of an image's name without the file extension

  Returns:
  None
  """
  #Create filename from image name
  fn = img + ".jpeg"
  #Get all bounding boxes for the image
  ann = annotations.query("filename == @fn")
  #Convert to array
  ann = np.array(ann)

  #Define XML root
  root = ET.Element("annotation")
  #XML folder, filename and path attributes
  folder = ET.SubElement(root, "folder").text = "train"
  filename = ET.SubElement(root, "filename").text = fn
  path = ET.SubElement(root, "path").text = "./drive/MyDrive/Tensorflow/workspace/training_demo/images/train/" + fn
  #Define source element
  source = ET.SubElement(root, "source")
  #Database element
  db = ET.SubElement(source, "database").text = "Note"
  #Height, width and depth of image
  size = ET.SubElement(root, "size")
  width = ET.SubElement(size, "width"). text = str(ann[0][6])
  height = ET.SubElement(size, "height").text = str(ann[0][7])
  depth = ET.SubElement(size, "depth").text = "3"
  #Segemented element
  segmented = ET.SubElement(root, "segmented").text = "0"
  #Create object eement with bounding box coordinates for each bounding box
  for bb in ann:
    obj = ET.SubElement(root, "object")
    name = ET.SubElement(obj, "name").text = "note"
    pose = ET.SubElement(obj, "pose").text = "Unspecified"
    trunc = ET.SubElement(obj, "truncated").text = "0"
    diff = ET.SubElement(obj, "difficult").text = "0"
    bbox = ET.SubElement(obj, "bndbox")
    xmin = ET.SubElement(bbox, "xmin").text = str(bb[1])
    ymin = ET.SubElement(bbox, "ymin").text = str(bb[2])
    xmax = ET.SubElement(bbox, "xmax").text = str(bb[3])
    ymax = ET.SubElement(bbox, "ymax").text = str(bb[4])
  
  #Create XML tree and write to file
  tree = ET.ElementTree(root)
  tree.write("./drive/MyDrive/annotations/" + img + ".xml")

## Create XML Files

In [ ]:
#Loop through image array and create an XML file for each image and its 30 augmented counter parts
#E.g: A.jpeg, A_1.jpeg, A_2.jpeg ... A_30.jpeg, B.jpeg ... XJ_30.jpeg
for img in images:
  create_xml(img)
  for x in range(1, 31):
    n = img + "_" + str(x)
    create_xml(n)